In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
from IPython.display import display
import seaborn as sns
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from collections import Counter, defaultdict
from operator import itemgetter
import tensorflow as tf
from tensorflow import keras
from pylab import rcParams
from pylab import savefig
import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
from lightfm.cross_validation import random_train_test_split
from scipy.sparse import csr_matrix
import scipy

from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score
import matplotlib.ticker as ticker
from math import sqrt
from sklearn.metrics import mean_squared_error

C:\Users\user\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
# Install Altair and activate its colab renderer.
#print("Installing Altair...")
#!pip install git+git://github.com/altair-viz/altair.git
#import altair as alt
#alt.data_transformers.enable('default', max_rows=None)
#alt.renderers.enable('colab')
#print("Done installing Altair.")

# Install spreadsheets and import authentication module.
#USER_RATINGS = False
#!pip install --upgrade -q gspread
#from google.colab import auth
#import gspread
#from oauth2client.client import GoogleCredentials

In [4]:
import matplotlib.pyplot as plt
import os
import warnings
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

Firstly, I converted my data files from .dat to .csv format. I did this via excel using the data tab and it's "get external data" option and extracted it from text. All the delimiting was done by default and I had my files in a delimited format.

In [41]:
df = pd.read_csv("data/hetrec2011-lastfm-2k/artists.csv")
df

id               name                                           url  \
0          1       MALICE MIZER         http://www.last.fm/music/MALICE+MIZER   
1          2    Diary of Dreams      http://www.last.fm/music/Diary+of+Dreams   
2          3  Carpathian Forest    http://www.last.fm/music/Carpathian+Forest   
3          4       Moi dix Mois         http://www.last.fm/music/Moi+dix+Mois   
4          5        Bella Morte          http://www.last.fm/music/Bella+Morte   
...      ...                ...                                           ...   
17627  18741    Diamanda GalÃ¡s  http://www.last.fm/music/Diamanda+Gal%C3%A1s   
17628  18742             Aya RL               http://www.last.fm/music/Aya+RL   
17629  18743        Coptic Rain          http://www.last.fm/music/Coptic+Rain   
17630  18744       Oz Alchemist         http://www.last.fm/music/Oz+Alchemist   
17631  18745   Grzegorz Tomczak     http://www.last.fm/music/Grzegorz+Tomczak   

                                              pictureURL  
0        http://userserve-ak.last.fm/serve/252/10808.jpg  
1      http://userserve-ak.last.fm/serve/252/3052066.jpg  
2      http://userserve-ak.last.fm/serve/252/40222717...  
3      http://userserve-ak.last.fm/serve/252/54697835...  
4      http://userserve-ak.last.fm/serve/252/14789013...  
...                                                  ...  
17627  http://userserve-ak.last.fm/serve/252/16352971...  
17628   http://userserve-ak.last.fm/serve/252/207445.jpg  
17629   http://userserve-ak.last.fm/serve/252/344868.jpg  
17630  http://userserve-ak.last.fm/serve/252/29297695...  
17631  http://userserve-ak.last.fm/serve/252/59486303...  

[17632 rows x 4 columns]

In [42]:
df['id'] = pd.to_numeric(df['id'])
lst = []
m = np.array(df['id'])
for i in range(0,17632):
    #print(i)
    if i not in df.id.values:
        lst.append(i)

len(lst)

965

In [43]:
newart = []
for i in range(0, 17632):
    newart.append(i)

In [44]:
newart = np.array(newart)
df['artID'] = newart.tolist()

In [45]:
#df.drop(columns=['id'], inplace=True)
df.head()

id               name                                         url  \
0   1       MALICE MIZER       http://www.last.fm/music/MALICE+MIZER   
1   2    Diary of Dreams    http://www.last.fm/music/Diary+of+Dreams   
2   3  Carpathian Forest  http://www.last.fm/music/Carpathian+Forest   
3   4       Moi dix Mois       http://www.last.fm/music/Moi+dix+Mois   
4   5        Bella Morte        http://www.last.fm/music/Bella+Morte   

                                          pictureURL  artID  
0    http://userserve-ak.last.fm/serve/252/10808.jpg      0  
1  http://userserve-ak.last.fm/serve/252/3052066.jpg      1  
2  http://userserve-ak.last.fm/serve/252/40222717...      2  
3  http://userserve-ak.last.fm/serve/252/54697835...      3  
4  http://userserve-ak.last.fm/serve/252/14789013...      4

There appears to be some unclean names such as "Diamanda GalÃ¡s" among others just taking an initial look at the data here.

In [62]:
df1 = pd.read_csv("data/hetrec2011-lastfm-2k/tags.csv")
df1

tagID           tagValue
0          1              metal
1          2  alternative metal
2          3          goth rock
3          4        black metal
4          5        death metal
...      ...                ...
11941  12644              suomi
11942  12645          symbiosis
11943  12646            sverige
11944  12647               eire
11945  12648     electro latino

[11946 rows x 2 columns]

In [63]:
df2 = pd.read_csv("data/hetrec2011-lastfm-2k/user_artists.csv")
df2

userID  artistID  weight
0           2        51   13883
1           2        52   11690
2           2        53   11351
3           2        54   10300
4           2        55    8983
...       ...       ...     ...
92829    2100     18726     337
92830    2100     18727     297
92831    2100     18728     281
92832    2100     18729     280
92833    2100     18730     263

[92834 rows x 3 columns]

In [64]:
df2['artistID'].min()

1

In [ ]:
# Since the ids start at 2, we get them to start at 0. We also need to have the max value 1 less than the no of unique values.
df2["userID"] = df2["userID"].apply(lambda x: str(x-2))
df2["artistID"] = df2["artistID"].apply(lambda x: str(x-1))

In [ ]:
df2['userID'] = df2['userID'].astype(int)
xyz =  np.array(df2['userID'])
#zzz = np.array(played['userID'])
vals = []
for i in range(len(xyz)):
    v = xyz[i]
    if v not in vals:
        vals.append(v)
    else:
        continue

In [ ]:
vals[-1]

In [ ]:
unique_list = list(set(vals))
unique_list.sort()
unique_list[-1]

In [ ]:
usenew = []
for i in range(0, 1892):
    usenew.append(i)

usenew[-1]

In [ ]:
keys = unique_list
values = usenew
dictionary = dict(zip(keys, values))
print(dictionary) # {'a': 1, 'b': 2, 'c': 3}

In [ ]:
s = df2['userID']

df2['userID'] = s.map(dictionary)

In [ ]:
df2.head()

In [ ]:
df2['artistID'] = df2['artistID'].astype(int)
xyz =  np.array(df2['artistID'])
#zzz = np.array(played['userID'])
vals = []
for i in range(len(xyz)):
    v = xyz[i]
    if v not in vals:
        vals.append(v)
    else:
        continue

In [ ]:
unique_list = list(set(vals))
unique_list.sort()
unique_list[0]

In [ ]:
usenew = []
for i in range(0, 17632):
    usenew.append(i)

usenew[-1]

In [ ]:
keys = unique_list
values = usenew
diction = dict(zip(keys, values))
#print(diction) # {'a': 1, 'b': 2, 'c': 3}

In [ ]:
s = df2['artistID']

df2['artistID'] = s.map(diction)

In [ ]:
df2['weight'].max()

In [ ]:
df2

In [ ]:
df3 = pd.read_csv("data/hetrec2011-lastfm-2k/user_friends.csv")
df3

In [ ]:
# Since the ids start at 2, we get them to start at 0. We also need to have the max value 1 less than the no of unique values.
df3["userID"] = df3["userID"].apply(lambda x: str(x-2))
df3["friendID"] = df3["friendID"].apply(lambda x: str(x-2))

df3['userID'] = pd.to_numeric(df3['userID'])
df3['friendID'] = pd.to_numeric(df3['friendID'])

In [ ]:
df3['friendID'].max()

In [ ]:
df3['friendID'].nunique()

In [ ]:
xyz =  np.array(df3['userID'])
#zzz = np.array(df2['userID'])
vals = []
for i in range(len(xyz)):
    if xyz[i] not in vals:
        vals.append(xyz[i])

In [ ]:
unique_list = list(set(vals))
unique_list.sort()
unique_list[-1]

In [ ]:
usenew = []
for i in range(0, 1892):
    usenew.append(i)

usenew[-1]

In [ ]:
keys = unique_list
values = usenew
dictionary = dict(zip(keys, values))
#print(dictionary) # {'a': 1, 'b': 2, 'c': 3}

In [ ]:
s = df3['userID']

df3['userID'] = s.map(dictionary)

In [ ]:
o = df3['friendID']

df3['friendID'] = o.map(dictionary)

In [ ]:
df3['friendID'].max()

In [ ]:
df3.isnull().values.any()

In [10]:
df4 = pd.read_csv("data/hetrec2011-lastfm-2k/user_taggedartists-timestamps.csv")
df4

userID  artistID  tagID  day  month  year
0            2        52     13    1      4  2009
1            2        52     15    1      4  2009
2            2        52     18    1      4  2009
3            2        52     21    1      4  2009
4            2        52     41    1      4  2009
...        ...       ...    ...  ...    ...   ...
186474    2100     16437      4    1      7  2010
186475    2100     16437    292    1      5  2010
186476    2100     16437   2087    1      7  2010
186477    2100     16437   2801    1      5  2010
186478    2100     16437   3335    1      7  2010

[186479 rows x 6 columns]

In [ ]:
df5 = pd.read_csv("data/hetrec2011-lastfm-2k/user_taggedartists.csv")
df5.head()

Our last two dataframes seem exactly the same. Let's check this before we delete anything.

In [ ]:
def checkequality(A, B):

    df11 = A.sort_index(axis=1)
    df11 = df11.sort_values(df11.columns.tolist()).reset_index(drop=True)

    df22 = B.sort_index(axis=1)
    df22 = df22.sort_values(df22.columns.tolist()).reset_index(drop=True)
    return (df11 == df22).values.all()

a = checkequality(df4, df5)
print (a)

Two of our files are exactly the same. We can delete one of these accordingly.

In [ ]:
del df5

In [ ]:
# Since the ids start at 2, we get them to start at 0. We also need to have the max value 1 less than the no of unique values.
df4["userID"] = df4["userID"].apply(lambda x: str(x-2))
df4["artistID"] = df4["artistID"].apply(lambda x: str(x-1))

df4['artistID'] = df4['artistID'].astype(int)
df4['userID'] = df4['userID'].astype(int)

In [ ]:
xyz =  np.array(df4['artistID'])
#zzz = np.array(played['userID'])
vals = []
for i in range(len(xyz)):
    v = xyz[i]
    if v not in vals:
        vals.append(v)
    else:
        continue

In [ ]:
unique_list = list(set(vals))
unique_list.sort()
print(unique_list[-1])

usenew = []
for i in range(0, 17632):
    usenew.append(i)

usenew[-1]

In [ ]:
keys = unique_list
values = usenew
diction = dict(zip(keys, values))
#print(diction) # {'a': 1, 'b': 2, 'c': 3}

In [ ]:
s = df4['artistID']

df4['artistID'] = s.map(diction)

In [ ]:
s = df4['userID']

df4['userID'] = s.map(dictionary)
print(dictionary)

In [ ]:
df4

In [ ]:
df4.drop(columns=['day', 'month', 'year'], inplace=True)

In [ ]:
tags = pd.merge(df1, df4, how="inner", left_on="tagID", right_on="tagID")
tags.isnull().values.any()
tags['artistID'].max()

# Methodology
Now that all our data files are read in and in the appropriate format we will begin our end to end process. These are as follows:
##### 1. Data cleaning and processing
##### 2. Visualization
##### 3. Fitting our Model
##### 4. Evaluating our Model

# Cleaning and Processing

### Initial analysis and cleaning

In [ ]:
df['name'].value_counts()

In [ ]:
df1['tagID'].value_counts()

After initial observation, I feel the timestamp dataframe is of little use to me and hence disregard it. As per the readme file we have 1892 users and 17632 artists in our data.

In [ ]:
dfs = [df, df1, df2, df3, df4]
na = []
for i in range(len(dfs)):
    if dfs[i].isnull().values.any() > 0:
        na.append(dfs[i])

In [ ]:
na

The only dataframe with nulls is our artists dataframe. Let's investigate this further to see if there is any important missing values such as ID's etc.

In [ ]:
df.dtypes

In [ ]:
features_with_na = [features for features in df.columns if df[features].isnull().sum() > 0]

for feature in features_with_na: 
    print(feature, np.round(df[feature].isnull().mean(), 4), '% missing values')
    print(features_with_na)

This is a positive result as there are very few null values in the dataframe and the small amount that exist are in a column of lesser important that we will not need to impute missing values for.

In [ ]:
played = pd.merge(df, df2, how="inner", left_on="artID", right_on="artistID")
played.rename(columns={"weight": "played"}, inplace=True)

We will drop the pictureURL column as there is not much information to be gained and there is some nulls present.

In [ ]:
played.drop(columns=['pictureURL'], inplace=True)

# Last Method

In [ ]:
ratings_df = played.pivot(
    index='userID', 
    columns='artistID', 
    values='playedUserNorm'
)

In [ ]:
ratings = ratings_df.fillna(0).values

In [ ]:
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('{:.2f}%'.format(sparsity))

In [ ]:
MIN_USER_RATINGS = 35
DELETE_RATING_COUNT = 15

def traintest_split(ratings):
    
    validation = np.zeros(ratings.shape)
    train = ratings.copy()
    
    for user in np.arange(ratings.shape[0]):
        if len(ratings[user,:].nonzero()[0]) >= MIN_USER_RATINGS:
            val_ratings = np.random.choice(
                ratings[user, :].nonzero()[0], 
                size=DELETE_RATING_COUNT,
                replace=False
            )
            train[user, val_ratings] = 0
            validation[user, val_ratings] = ratings[user, val_ratings]
    return train, validation

In [ ]:
train, val = traintest_split(ratings)

In [ ]:
train.shape

In [ ]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [ ]:
class Recommender:
  
  def __init__(self, n_epochs=200, n_latent_features=3, lmbda=0.1, learning_rate=0.001):
    self.n_epochs = n_epochs
    self.n_latent_features = n_latent_features
    self.lmbda = lmbda
    self.learning_rate = learning_rate
  
  def predictions(self, P, Q):
    return np.dot(P.T, Q)
  
  def fit(self, X_train, X_val):
    m, n = X_train.shape

    self.P = 3 * np.random.rand(self.n_latent_features, m)
    self.Q = 3 * np.random.rand(self.n_latent_features, n)
    
    self.train_error = []
    self.val_error = []

    users, items = X_train.nonzero()
    
    for epoch in range(self.n_epochs):
        for u, i in zip(users, items):
            error = X_train[u, i] - self.predictions(self.P[:,u], self.Q[:,i])
            self.P[:, u] += self.learning_rate * (error * self.Q[:, i] - self.lmbda * self.P[:, u])
            self.Q[:, i] += self.learning_rate * (error * self.P[:, u] - self.lmbda * self.Q[:, i])

        train_rmse = rmse(self.predictions(self.P, self.Q), X_train)
        val_rmse = rmse(self.predictions(self.P, self.Q), X_val)
        self.train_error.append(train_rmse)
        self.val_error.append(val_rmse)
        
    return self
  
  def predict(self, X_train, user_index):
    y_hat = self.predictions(self.P, self.Q)
    predictions_index = np.where(X_train[user_index, :] == 0)[0]
    return y_hat[user_index, predictions_index].flatten()

In [ ]:
recommender = Recommender().fit(train, val)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(recommender.n_epochs), recommender.train_error, marker='o', label='Training Data');
plt.plot(range(recommender.n_epochs), recommender.val_error, marker='v', label='Validation Data');
plt.xlabel('Number of Epochs');
plt.ylabel('RMSE');
plt.legend()
plt.grid()
plt.show()

## Making Recommendations

In [ ]:
user_id = 83
user_index = ratings_df.index.get_loc(user_id)
predictions_index = np.where(train[user_index, :] == 0)[0]

rating_predictions = recommender.predict(train, user_index)

In [ ]:
def create_artist_ratings(artists_df, artists_index, ratings, n=10):
    artist_ids = ratings_df.columns[artists_index]
    artist_ratings = pd.DataFrame(data=dict(artistId=artist_ids, rating=ratings))
    top_n_artists = artist_ratings.sort_values("rating", ascending=False).head(n)
  
    artist_recommendations = artists_df[artists_df.artID.isin(top_n_artists.artistId)].reset_index(drop=True)
    artist_recommendations['rating'] = pd.Series(top_n_artists.rating.values)
    return artist_recommendations.sort_values("rating", ascending=False)

In [ ]:
existing_ratings_index = np.where(train[user_index, :] > 0)[0]
existing_ratings = train[user_index, existing_ratings_index]

create_artist_ratings(df, existing_ratings_index, existing_ratings)

In [ ]:
create_artist_ratings(df, predictions_index, rating_predictions)

Let's run a similar evaluation process over our second alternate method. We will look at the same user ID of 83 and look at the recommendations it makes.

In [ ]:
played[played['userID'] == 83]

In [ ]:
pre = create_artist_ratings(df, predictions_index, rating_predictions)
zzzz2 = pd.merge(pre, tags, how="inner", left_on=['artID'], right_on=['artistID'])

In [ ]:
zzzz2['tagValue'].value_counts()

We can see again that our system appears to be recommending artists of popular genres / tags more so and not making relevant recommendations. It is good that it is making novel recommendations on one hand and giving the user a wider scope of music. However, it must be said it would be more advisable if it could make recommendations more similar to what a user listens to identifying less popular artists of similar genres that they may have missed. This recommender like our neural network recommender appears to be suffering from the "cold start" problem many recommenders encounter.

In [68]:
## rough work

In [ ]:
df2['userID'] = df2['userID'].astype(int)
df2['artistID'] = df2['artistID'].astype(int)

In [12]:
vals = (df4['artistID'].unique()).tolist()

In [13]:
vals.sort()

In [27]:
vals = (df4['artistID'].unique()).tolist()
vals.sort()
values = []
for i in range(len(vals)):
    if vals[i] in m:
        values.append(vals[i])

In [28]:
len(df4['artistID'].unique())

12523

In [29]:
df4['artistID'].max()

18744

In [30]:
df4.dtypes

userID      int64
artistID    int64
tagID       int64
day         int64
month       int64
year        int64
dtype: object

In [31]:
vals[-1]

18744

In [32]:
len(values)

12133

In [34]:
missing = []
for i in range(len(vals)):
    if vals[i] not in values:
        missing.append(vals[i])

In [35]:
len(missing)

390

In [36]:
missing[0]

14

In [40]:
list3 = values + missing
print(len(list3))

12523


In [46]:
keys = m
values = newart
dictionary = dict(zip(keys, values))
print(dictionary) # {'a': 1, 'b': 2, 'c': 3}

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 15: 13, 16: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19, 22: 20, 23: 21, 24: 22, 25: 23, 26: 24, 27: 25, 28: 26, 30: 27, 31: 28, 32: 29, 34: 30, 36: 31, 37: 32, 38: 33, 39: 34, 40: 35, 41: 36, 43: 37, 44: 38, 45: 39, 46: 40, 47: 41, 48: 42, 49: 43, 50: 44, 51: 45, 52: 46, 53: 47, 54: 48, 55: 49, 56: 50, 57: 51, 58: 52, 59: 53, 60: 54, 61: 55, 62: 56, 63: 57, 64: 58, 65: 59, 66: 60, 67: 61, 68: 62, 69: 63, 70: 64, 71: 65, 72: 66, 73: 67, 74: 68, 75: 69, 76: 70, 77: 71, 78: 72, 79: 73, 80: 74, 81: 75, 82: 76, 83: 77, 84: 78, 85: 79, 86: 80, 87: 81, 88: 82, 89: 83, 90: 84, 91: 85, 92: 86, 93: 87, 94: 88, 95: 89, 96: 90, 97: 91, 98: 92, 99: 93, 100: 94, 101: 95, 102: 96, 103: 97, 104: 98, 105: 99, 106: 100, 107: 101, 108: 102, 109: 103, 110: 104, 111: 105, 112: 106, 113: 107, 114: 108, 115: 109, 116: 110, 117: 111, 118: 112, 119: 113, 120: 114, 121: 115, 122: 116, 123: 117, 124: 118, 125: 119, 126: 120, 12

In [72]:
a_subset = {key: value for key, value in dictionary.items() if key in list3}

In [73]:
print(len(a_subset))

12133


In [53]:
df4[df4['artistID'] == 14]

userID  artistID  tagID  day  month  year
3267       40        14     61    1      9  2008
3268       40        14     86    1      9  2008
55420     557        14     10    1     10  2009
55421     557        14     12    1     10  2009
55422     557        14    565    1     10  2009

In [54]:
df[df['id'] == 14]

Empty DataFrame
Columns: [id, name, url, pictureURL, artID]
Index: []

In [55]:
df4 = df4[~df4.artistID.isin(missing)]

In [57]:
df4[df4['artistID'] == 14]

Empty DataFrame
Columns: [userID, artistID, tagID, day, month, year]
Index: []

In [59]:
s = df4['artistID']

df4['artistID'] = s.map(a_subset)

C:\Users\user\AppData\Local\Temp/ipykernel_11332/3743877304.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['artistID'] = s.map(a_subset)


In [65]:
df4['userID'].nunique()

1892

In [67]:
df4['artistID'].min()

0